In [67]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import traitlets
from IPython.display import display
from ipywidgets import widgets
from tkinter import Tk, filedialog
import pandas as pd
from IPython.display import clear_output
from tqdm import tqdm
import pandas as pd
import io
from ipywidgets import interact, interactive, fixed, interact_manual
import os
from ipywidgets import AppLayout, Button, Layout

# Import libraries
from APfuncs import *
from ipywidgets import *
from IPython.display import display, Javascript
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline                    # Safe smooth interpolation
from scipy.signal import find_peaks
from scipy.signal import medfilt
from scipy import signal


import peakutils
from peakutils.plot import plot as pplot
from matplotlib.animation import FuncAnimation
import matplotlib

from scipy import stats

from numpy.fft import fft, fftfreq, ifft

import glob

from tqdm import tqdm

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from renishawWiRE import WDFReader

import selenium
import os
from selenium import webdriver

In [10]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

In [14]:
def make_app_layout(h, ls, c, f):

    return AppLayout(header=h,
                     left_sidebar=ls,
                     center=c,
                     right_sidebar=None,
                     footer=f,
                     pane_widths=[2, 3, 0],
                     pane_heights=[0.5, 5, 0.5])

In [11]:
    
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self, *args, **kwargs):
        """Initialize the SelectFilesButton class."""
        super(SelectFilesButton, self).__init__(*args, **kwargs)
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select File"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)
        
        

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.
        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)
        f = b.files
        '''
        for i in range(0,len(f)):
            print(i,f[i],end="\n")   # load preview cut process plot
        '''
        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [79]:
def filter_median(data,k_size):
    
    filt_data = medfilt(data,kernel_size=k_size.value)
 
    #plt.figure()
    #plt.plot(x_data,filt_data,label='Median Filtered Data')
    #plt.plot(wvl,y_data/np.max(y_data),label='Actual Data')
    #plt.plot(peak_x,peak_y/np.max(intens_data),'ob')
    #plt.title(fdata[i])
    #plt.legend()
    #plt.show()
    
    return filt_data

In [75]:
def filter_savgol(data,k_size,p_order):
    
    data[np.isnan(data)] = 0
    print('yaya')
    
    sav_filt_data=signal.savgol_filter(data,k_size.value,p_order.value)
    
    return sav_filt_data

In [90]:
#plt.rcParams.update({'figure.max_open_warning': 0})
plt.close('all')

class Data_Plotter(widgets.HBox):
    def __init__(self):
        super().__init__()
        output = widgets.Output()
        
        style = {'description_width': 'initial'}

        self.x = []
        self.y = []
        initial_color = '#FF00DD'

        with output:
            self.fig, self.ax = plt.subplots(constrained_layout=True,
                                             figsize=(5, 3.5))
            #plt.autoscale(enable=True, axis='both')
        self.line, = self.ax.plot(self.x, self.y, initial_color)

        self.fig.canvas.toolbar_position = 'bottom'
        self.ax.grid(True)

        ### define plot data widgets

        color_picker = widgets.ColorPicker(value=initial_color,
                                           description='pick a color')

        text_xlabel = widgets.Text(value='',
                                   description='xlabel',
                                   continuous_update=False)

        text_ylabel = widgets.Text(value='',
                                   description='ylabel',
                                   continuous_update=False)

        x_lim = widgets.FloatRangeSlider(
            value=[500, 700],
            min=0,
            max=1000.0,
            step=0.1,
            description='X-axis range:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
        )

        y_lim = widgets.FloatRangeSlider(
            value=[0, 500],
            min=0,
            max=1000,
            step=0.1,
            description='Y-axis range:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
        )

        selectHalLamp_1 = SelectFilesButton()

        self.plot_data = widgets.Button(
            description='Plot Data',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Data plotted',
            icon='check'  # (FontAwesome names without the `fa-` prefix)
        )

        header_button = widgets.Button(description='Welcome to Data Plotter',
                                       button_style='success',
                                       layout=Layout(height='auto',
                                                     width='auto'))
        footer_button = widgets.Button(
            description='Created by Chaman Gupta, PZ Group, UW',
            button_style='success',
            layout=Layout(height='auto', width='auto'))

        #tab_contents = ['P0', 'P1', 'P2', 'P3', 'P4']
        #children_tab = [widgets.Text(description=name) for name in tab_contents]

        
        controls = widgets.VBox([
            color_picker,
            text_xlabel,
            text_ylabel,
            x_lim,
            y_lim,
            selectHalLamp_1,
            self.plot_data,
        ])

        controls.layout = make_box_layout()

        out_box = widgets.Box([output])
        output.layout = make_box_layout()
        
        ### define data processing widgets
        
        self.intes_data_check = widgets.Checkbox(value=False,description='Clean lamp data',disabled=False,indent=False)
         
        self.min_val_intensity = widgets.BoundedFloatText(value=550,min=0,max=1000,step=0.1,description='Minimum Intensity',disabled=False,style=style)
        self.med_check_w=widgets.Checkbox(value=False,description='Median Filtering',disabled=False,indent=False)
        self.savgol_check_w=widgets.Checkbox(value=False,description='Savgol Filtering',disabled=False,indent=False)
        self.clean_check_w=widgets.Checkbox(value=False,description='Data Cleaning',disabled=False,indent=False)
        
        self.selectHalLamp_1 = SelectFilesButton()
        
        self.k_size_med = widgets.IntSlider(value=3,min=1,max=11,step=2,description='Kernel Size',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
        self.k_size_savgol = widgets.IntSlider(value=11,min=1,max=53,step=2,description='Kernel Size',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
        self.p_order_savgol = widgets.IntSlider(value=1,min=1,max=11,step=2,description='Polynomial Order',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')

        self.data_proc = widgets.VBox([self.plot_data,self.intes_data_check])
        self.data_proc.layout = make_box_layout()
    
        
        tab = widgets.Tab()
        tab.titles = ('Plot Controls','Data Cleaning')
        tab.children = [controls,self.data_proc]
        

        # observe stuff
        #int_slider.observe(self.update, 'value')
        color_picker.observe(self.line_color, 'value')
        text_xlabel.observe(self.update_xlabel, 'value')
        text_ylabel.observe(self.update_ylabel, 'value')
        x_lim.observe(self.update_xlim, 'value')
        y_lim.observe(self.update_ylim, 'value')
        selectHalLamp_1.on_click(self.get_data)
        self.plot_data.on_click(self.update_selectf)
        self.intes_data_check.observe(self.f,'value')

        text_xlabel.value = 'x'
        text_ylabel.value = 'y'

        main_app = make_app_layout(header_button, tab, output,footer_button)

        # add to children
        self.children = [main_app]

    def get_data(self, selectHalLamp_1):

        flamp_1 = selectHalLamp_1.files

        #print(flamp_1)

        for i in range(len(flamp_1)):

            head, tail = os.path.split(flamp_1[i])

            data = pd.read_csv(flamp_1[i],
                               sep=None,
                               engine='python',
                               names=['col_a_data', 'col_b_data'])
            self.x_data = data['col_a_data'].to_numpy()
            self.y_data = data['col_b_data'].to_numpy()

            self.x = x_data
            self.y = y_data

            #print(self.x)
            #print(self.y)

    def line_color(self, change):
        self.line.set_color(change.new)

    def update_xlabel(self, change):
        self.ax.set_xlabel(change.new)

    def update_ylabel(self, change):
        self.ax.set_ylabel(change.new)

    def update_xlim(self, change):
        self.ax.set_xlim(change.new)

    def update_ylim(self, change):
        self.ax.set_ylim(change.new)

    def update_selectf(self,plot_data):
        
        [l.remove() for l in self.ax.lines]
        self.line, = self.ax.plot(self.x, self.y, color='g')
        plt.autoscale(enable=True, axis='both')        
        self.fig.canvas.draw()
        
    def f1(self,change): ## Median Filtering

        if change.new==True:
            self.med_vbox.children = tuple(list(self.med_vbox.children) + [self.k_size_med])
            self.y = filter_median(self.y_data,self.k_size_med)
            
            self.line, = self.ax.plot(self.x, self.y, color='g')
            plt.autoscale(enable=True, axis='both')        
            self.fig.canvas.draw()
        
        elif change.new==False:
            self.med_vbox.children = [self.med_check_w]
            
    def f2(self,change): ## Savgol Filtering
        if change.new==True:
            self.savgol_vbox.children = tuple(list(self.savgol_vbox.children) + [self.k_size_savgol,self.p_order_savgol])
            self.y = filter_savgol(self.y,self.k_size_savgol,self.p_order_savgol)
            
            [l.remove() for l in self.ax.lines]
            self.line, = self.ax.plot(self.x, self.y, color='k')
            plt.autoscale(enable=True, axis='both')
            self.fig.canvas.draw()
            
        elif change.new == False:
            self.savgol_vbox.children = [self.savgol_check_w]
        
  
    def f(self,change):
        if change.new==True:
            style = {'description_width': 'initial'}
            
            self.med_vbox = widgets.VBox([self.med_check_w])
            self.savgol_vbox = widgets.VBox([self.savgol_check_w])
            self.clean_vbox = widgets.VBox([self.clean_check_w])
            self.sel_vbox = widgets.VBox([self.selectHalLamp_1])
            
            
            self.data_proc.children=tuple(list(self.data_proc.children) + [self.min_val_intensity,self.med_vbox,self.savgol_vbox,self.clean_vbox,self.sel_vbox]) 
        
            self.med_check_w.observe(self.f1,'value')
            self.savgol_check_w.observe(self.f2,'value')
            
        elif change.new == False:
            
            self.data_proc.children = [self.plot_data,self.intes_data_check]
            
            
Data_Plotter()

Data_Plotter(children=(AppLayout(children=(Button(button_style='success', description='Welcome to Data Plotter…

NameError: name 'x_data' is not defined